# TinyTag

In [1]:
import os
import json
import numpy as np
from datetime import datetime
from tinytag import TinyTag
from bson import ObjectId
from pymongo import MongoClient
from pprint import PrettyPrinter
pp = PrettyPrinter()

In [2]:
MUSIC_DIR = "/Users/elizabethchen/Music/Muse"

In [3]:
for artist_dir in os.listdir(MUSIC_DIR):
    if not artist_dir.startswith(".") or artist_dir.endswith(".jpg"):
        print(os.path.join(MUSIC_DIR, artist_dir))

/Users/elizabethchen/Music/Muse/Abandoning Sunday
/Users/elizabethchen/Music/Muse/Maude Latour
/Users/elizabethchen/Music/Muse/Dexys Midnight Runners
/Users/elizabethchen/Music/Muse/Halsey
/Users/elizabethchen/Music/Muse/As It Is
/Users/elizabethchen/Music/Muse/Demi Lovato
/Users/elizabethchen/Music/Muse/Eminem
/Users/elizabethchen/Music/Muse/Meg Myers
/Users/elizabethchen/Music/Muse/Kelly Clarkson
/Users/elizabethchen/Music/Muse/JoJo
/Users/elizabethchen/Music/Muse/Arctic Monkeys
/Users/elizabethchen/Music/Muse/benny blanco
/Users/elizabethchen/Music/Muse/The Killers
/Users/elizabethchen/Music/Muse/Manchester Orchestra
/Users/elizabethchen/Music/Muse/Cage the Elephant
/Users/elizabethchen/Music/Muse/Diddy - Dirty Money
/Users/elizabethchen/Music/Muse/Loote
/Users/elizabethchen/Music/Muse/Bastille
/Users/elizabethchen/Music/Muse/Jonas Brothers
/Users/elizabethchen/Music/Muse/Olivia Rodrigo
/Users/elizabethchen/Music/Muse/Anne-Marie
/Users/elizabethchen/Music/Muse/Katharine McPhee & Jer

## TinyTag.get(song_file) attributes
* Of interest:
```
tag.title         # title of the song
tag.duration      # duration of the song in seconds
tag.artist        # artist name as string
tag.album         # album as string
tag.albumartist   # album artist as string
tag.track         # track number as string
tag.genre         # genre as string
tag.year          # year or data as string
```
* ALL:
```
tag.album         # album as string
tag.albumartist   # album artist as string
tag.artist        # artist name as string
tag.audio_offset  # number of bytes before audio data begins
tag.bitrate       # bitrate in kBits/s
tag.comment       # file comment as string
tag.composer      # composer as string 
tag.disc          # disc number
tag.disc_total    # the total number of discs
tag.duration      # duration of the song in seconds
tag.filesize      # file size in bytes
tag.genre         # genre as string
tag.samplerate    # samples per second
tag.title         # title of the song
tag.track         # track number as string
tag.track_total   # total number of tracks as string
tag.year          # year or data as string
```

In [4]:
for artist_dir in os.listdir(MUSIC_DIR):
    if not artist_dir.startswith("."):
        #print('Now showing contents of %s' % os.path.join(MUSIC_DIR, artist_dir))
        for album_dir in os.listdir(os.path.join(MUSIC_DIR, artist_dir)):
            if not album_dir.startswith("."):
                if not album_dir.endswith(".jpg"):
                    for song in os.listdir(os.path.join(MUSIC_DIR, artist_dir, album_dir)):
                        if not song.startswith("."):
                            if not song.endswith(".jpg"):
                                tag = TinyTag.get(os.path.join(MUSIC_DIR, artist_dir, album_dir, song))
                                minutes = int(tag.duration//60)
                                seconds = int((tag.duration/60 - minutes)*60)
                                if seconds < 10:
                                    seconds = str(0) + str(seconds)
                                #print(f"""
                                #       Track Name: {tag.title} ({minutes}:{seconds})
                                #       Album: {tag.album}
                                #       Artist: {tag.artist}
                                #       Album Artist: {tag.albumartist}
                                #       Release Date: {tag.year}
                                #       """
                                #     )
        #print("\n")

# MongoDB

In [5]:
connection = MongoClient("mongodb://127.0.0.1:27017/museplayer" )
db = connection.museplayer

### Artists Collection Structure

```
Artists: {
            _id: ObjectID
            artistName: string
            albums: [ list of album_ids ]
            profilePic: [image] or [filename/location]
         }
```

### Albums Collection Structure

```
Albums: { 
           _id: ObjectId
           albumName: string
           albumArtist: string
           releaseDate: [date or year]
           albumCover: [image] or [filename/location]
           genre: string
           songs: [
                      {
                         _id: ObjectID
                         title: string
                         duration: float
                         artist: string
                         isFavorited: boolean
                         numPlays: integer
                      }
                   ]
```

In [6]:
albums = db.albums
albums.drop()

artists = db.artists
artists.drop()

songs = db.songs
songs.drop()

for artist_name in os.listdir(MUSIC_DIR):
    if not artist_name.startswith("."):
        artist_album_ids = []
        for album_name in os.listdir(os.path.join(MUSIC_DIR, artist_name)):
            if not album_name in {'profile.jpg', '.DS_Store'}: # Two files to exclude when searching the artist directory
                cover_loc = os.path.join(os.path.join(MUSIC_DIR, artist_name, album_name, 'cover.jpg'))
                album_id =  ObjectId()
                artist_id = ObjectId()
                songs_list = []
                for song in os.listdir(os.path.join(MUSIC_DIR, artist_name, album_name)):
                    if not song in {'cover.jpg', '.DS_Store'}: # Two files to exclude when searching the albums directory
                        try:
                            file_loc = os.path.join(MUSIC_DIR, artist_name, album_name, song)
                            tag = TinyTag.get(file_loc)                    
                            song_entry = {
                                '_id': ObjectId(),
                                'artistId': artist_id,
                                'albumId': album_id,
                                'songFileLoc': file_loc,
                                'title': tag.title,
                                'duration': np.round(tag.duration, 1),
                                'trackNumber': tag.track,
                                'artistName': tag.artist,
                                'albumName': tag.album,
                                'albumArtist': tag.albumartist,
                                'isFavorited': False,
                                'numPlays': 0
                            }
                            songs_list.append(song_entry)
                            songs.insert_one(song_entry)
                        except:
                            print(os.path.join(MUSIC_DIR, artist_name, album_name, song))
                artist_album_ids.append(album_id)
                album_entry  = {
                    '_id': album_id,
                    'artist_id': artist_id,
                    'albumName': tag.album,
                    'albumArtist': tag.albumartist,
                    'releaseDate': tag.year,
                    'genre': tag.genre,
                    'songs': songs_list,
                    'albumArtworkLoc': cover_loc,
                    'isFavorited': False
                }
                albums.insert_one(album_entry)       
        profile_location = os.path.join(MUSIC_DIR, artist_name, 'profile.jpg')
        artist_entry = {
            '_id': artist_id,
            'artistName': tag.albumartist,
            'artistAlbums': artist_album_ids,
            'profilePicLoc': profile_location,
            'isFavorited': False
        }
        artists.insert_one(artist_entry)

In [7]:
artists.create_index('artistName')
songs.create_index('title')
albums.create_index('albumName')

'albumName_1'